In [1]:
# Importing the required modules 
import os
import sys
import pandas as pd
import numpy as np

### Инвестора

In [2]:
rounds = pd.read_csv('rounds.csv')
rounds = rounds.drop(['Index'], axis=1)

ipos = pd.read_csv('ipos.csv')
ipos = ipos.drop(['Index', 'OrigID', 'RowNumber'], axis=1)

acquisitions = pd.read_csv('acquisitions.csv')
acquisitions = acquisitions.drop(['Inde', 'OrigID', 'RowNumber'], axis=1)

closed = pd.read_csv('closed.csv')
closed = closed.drop(['Index', 'OrigID', 'RowNumber'], axis=1)

In [3]:
priced_ipos = ipos.loc[~ipos['Valuation at IPO'].isin([np.nan])]
priced_acquisitions = acquisitions.loc[~acquisitions['Price'].isin([np.nan])]

In [4]:
ipo_acq_rounds = rounds.loc[(rounds['Organization Name'].isin(priced_ipos['Organization Name'].values)) | (rounds['Organization Name'].isin(priced_acquisitions['Acquiree Name'].values))]
a = ipo_acq_rounds[['Announced Date', 'Money Raised', 'Lead Investors', 'Round_Code']]
a = a.loc[~a['Lead Investors'].isin([np.nan])]
a

,Announced Date,Money Raised,Lead Investors,Round_Code
80,28/10/2022,570000000,Stonepeak,/funding_round/coresite-private-equity--8f7b9f7f
336,04/10/2022,155000000,"Koch Strategic Platforms,Siris Capital Group",/funding_round/mavenir-private-equity--791481c1
676,08/09/2022,12300000,"Framework Ventures,Polychain",/funding_round/slide-daf4-seed--0e64c0f3
892,18/08/2022,100000000,SK,/funding_round/atom-power-inc-series-c--1d47dd9d
1099,29/07/2022,287500000,The Sterling Group,/funding_round/ergotron-private-equity--9fc1f127
...,...,...,...,...
67193,01/11/1998,2250000,Innogenetics,/funding_round/cepheid-series-c--cf343d0a
67221,24/07/1998,9000000,New Enterprise Associates,/funding_round/good-technology-series-c--e00506cd
67244,26/05/1998,3000000,Lazard Technology Partners,/funding_round/cyveillance-series-a--3aab7c77
67324,03/08/1997,15000000,Jerusalem Global Ventures,/funding_round/oridion-series-a--ad32986a


In [5]:
b = pd.DataFrame(a['Lead Investors'].str.split(',').tolist(), index=a['Round_Code']).stack()
b = b.reset_index()[[0, 'Round_Code']] # var1 variable is currently labeled 0
b.columns = ['Investor', 'Round_Code'] # renaming var1
b = b.merge(a, how = 'left', on = 'Round_Code')
b

,Investor,Round_Code,Announced Date,Money Raised,Lead Investors
0,Stonepeak,/funding_round/coresite-private-equity--8f7b9f7f,28/10/2022,570000000,Stonepeak
1,Koch Strategic Platforms,/funding_round/mavenir-private-equity--791481c1,04/10/2022,155000000,"Koch Strategic Platforms,Siris Capital Group"
2,Siris Capital Group,/funding_round/mavenir-private-equity--791481c1,04/10/2022,155000000,"Koch Strategic Platforms,Siris Capital Group"
3,Framework Ventures,/funding_round/slide-daf4-seed--0e64c0f3,08/09/2022,12300000,"Framework Ventures,Polychain"
4,Polychain,/funding_round/slide-daf4-seed--0e64c0f3,08/09/2022,12300000,"Framework Ventures,Polychain"
...,...,...,...,...,...
5964,Innogenetics,/funding_round/cepheid-series-c--cf343d0a,01/11/1998,2250000,Innogenetics
5965,New Enterprise Associates,/funding_round/good-technology-series-c--e00506cd,24/07/1998,9000000,New Enterprise Associates
5966,Lazard Technology Partners,/funding_round/cyveillance-series-a--3aab7c77,26/05/1998,3000000,Lazard Technology Partners
5967,Jerusalem Global Ventures,/funding_round/oridion-series-a--ad32986a,03/08/1997,15000000,Jerusalem Global Ventures


In [16]:
b['Announced Date'] = pd.to_datetime(b['Announced Date'], errors='coerce')
tab = pd.crosstab(index = b['Investor'], columns = b['Announced Date'].dt.strftime('%Y/%m'), values = b['Money Raised'], aggfunc='sum', dropna = False, margins = True).sort_values('All', ascending=False)
tab.to_csv('investors.csv', encoding = 'utf-8-sig')
tab

Announced Date,1997/01,1997/03,1998/01,1998/05,1998/07,1999/01,1999/02,1999/05,1999/06,1999/08,...,2022/02,2022/03,2022/04,2022/05,2022/06,2022/07,2022/08,2022/10,2022/11,All
Investor,,,,,,,,,,,,,,,,,,,,,
All,8000000.0,15000000.0,10249996.0,3000000.0,9000000.0,56000000.0,13600000.0,22000000.0,18450000.0,6100000.0,...,543100000.0,30000000.0,520000000.0,65175000.0,630000000.0,287500000.0,154600000.0,570000000.0,30000000.0,319516400533
Warburg Pincus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11395060000
T. Rowe Price,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11123000000
SoftBank Vision Fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7220300000
Tiger Global Management,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5605018686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Nyca Partners,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000000
Holdinvest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000000
TMT Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000000
